In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key="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",
)

In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an expert at identifying table of contents from the given paragraph {paragraph} 
    Don't give any additional information or summary
    You must add the sub category but don't add 'The Official Gazette' and which edition

    ### The output must be in the following format: 
    Dictonary, each dictonary key should be main topics like Executive Council Resolutions, Laws and value should be list of sub topics
    Do not include Pages

    Sample output format
    
    {{
        "Laws": [
            "Law No. (2) of 2022 Concerning the Reorganisation of the National Rehabilitation Centre - Abu Dhabi"
        ],
        "Executive Council Resolutions": [
            "Chairman of the Executive Council Resolution No. (3) of 2022 Concerning the Appointment of the Director-General for Tourism at the Department of Culture and Tourism"
        ],
        "Circulars": [
            "Circular No. (1) of 2022 Concerning the Policy of Providing an Easy and Effortless Customer Experience in the Emirate of Abu Dhabi"
        ],
            "Decrees": [
            "Amiri Decree No. (13) of 2023 Concerning Sending a Judge to Retirement"
        ]
    }}
    
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """
)

question_router = prompt | llm


In [3]:
import os
import re
import json
from PyPDF2 import PdfReader

def remove_dot(text):
    return text[:-1] if text.endswith('.') else text

def extrating_content(key_topics, output_lower, final_output):

    for i in range(len(key_topics)):
        current_topic = key_topics[i]
        
        current_topic = remove_dot(current_topic)
        current_topic_lower = current_topic.lower()

        print("===========================================")
        current_topic = current_topic_lower.replace("'", '"')
        
        print(f"Current Topic: {current_topic}")

        if i < len(key_topics) - 1:
            next_topic = key_topics[i + 1]
            next_topic = remove_dot(next_topic)
            next_topic_lower = next_topic.lower()
            
            next_topic = next_topic_lower.replace("'", '"')
            print(f"Next Topic: {next_topic}")

            start = output_lower.find(current_topic)
            end = output_lower.find(next_topic)
            
            print(f"String index of current topic: {start}")
            print(f"String index of next topic: {end}")

            substring = output_lower[start:end]
        else:
            # For the last topic, take everything from the current topic to the end
            start = output_lower.find(current_topic) + len(current_topic)
            end = len(output_lower)  # Go to the end of the string
            substring = output_lower[start:end]

        print("\n")
        print(substring)

        final_output[current_topic] = substring
    
    return final_output

def table_of_content(reader):
    for page_no in range(1, 4):
        page = reader.pages[page_no]
        text = page.extract_text()
        return text

def parse_document(pdf_path):
    with open(pdf_path, 'rb') as file:

        key_topics = list()

        reader = PdfReader(file)
        num_pages = len(reader.pages)

        index_text = table_of_content(reader)
        index_data = question_router.invoke({"paragraph": index_text})
        output = ""
        for page_no in range(3, num_pages):
            page = reader.pages[page_no]
            text = page.extract_text()
            if text and text[0].isdigit():
                text = text[1:]
                text = re.sub(r'^\d+', '', text)

            cleaned_text = re.sub(r'[\n\r\t]', ' ', text)
            cleaned_text = cleaned_text.replace("  ", " ")
            cleaned_text = cleaned_text.replace("  ", " ")

            output = output + " " + cleaned_text
            output = output.strip()
        
        print("TOC\n")
        print(index_data.content)
        toc = json.loads(index_data.content)

        for key, value in toc.items():
            key_topics.extend(value)
        
        return toc, key_topics, output
    

In [5]:
directory_path = '../../doc/test/'
final_output = {}

for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'): 
        file_path = os.path.join(directory_path, filename) 
        print(f"Reading {file_path}...")
        
        toc, key_topics, output = parse_document(file_path)

        final_output = extrating_content(key_topics, output.lower(), final_output)

        

Reading ../../doc/test/First Edition English 2023.pdf...
TOC

{
    "Laws": [
        "Law No. (16) of 2022 Concerning The Licensing of The Drivers and Vehicles in The Emirate of Abu Dhabi.",
        "Law No. (1) of 2023 Concerning the establishment of Abu Dhabi Transport Company “Public Joint Stock Company”.",
        "Law No. (4) of 2023 Concerning The Establishment of Abu Dhabi Media Office."
    ],
    "Decrees": [
        "Amiri Decree No. (3) of 2023 Concerning Some Members of the Executive Council in the Emirate of Abu Dhabi."
    ],
    "Executive Council Resolutions": [
        "Executive Council Resolution No. (169) of 2022 Concerning the Thirty-First Investment Zone in the Emirate of Abu Dhabi.",
        "Executive Council Resolution No. (182) of 2022 Concerning the General Policy for the Management, Regulation and Protection of Groundwater in the Emirate of Abu Dhabi.",
        "Executive Council Resolution No. (183) of 2022 Concerning the Amendment of Some Provisions of Ch

: 

In [55]:
prompt = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id>
    You are an expert in identifying laws and resolutions from the provided document.

    The document references {index} and contains the following content: {paragraph}

    Instructions:

    Provide only references without additional information or summaries.
    Format the output as a Python list, e.g., ["resolution no. (106) of 2021 on economic licensing fees", "chairman of the executive council resolution no. (4) of 2021 concerning the reformation of the board of directors of abu dhabi housing authority", "law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments"].
    Make sure that you are including the resolution and law concerning in short summary
    Exclude self-reference {index} from the output.
    Present only law or reference numbers and their years.
    If a reference is not a law or resolution, ignore it and keep the empty Python list, e.g., []
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["index", "paragraph"],
)

reference_router = prompt | llm


In [56]:
#summary_text = final_output['executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities']

for index, content in final_output.items():
    response = reference_router.invoke(
        {
            "index": index,
            "paragraph": content
        }
    )
    print(index)
    print(response.content)
    print(content)

executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities
["chairman of the executive council resolution no. (27) of 2020 concerning the formation of the board of trustees of mohamed bin zayed university for humanities", "law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments"]
executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities having reviewed chairman of the executive council resolution no. (27) of 2020 concerning the formation of the board of trustees of mohamed bin zayed university for humanities. the executive council has decided the following: 1. the board of trustees of mohamed bin zayed university for humanities shall be reformed under the chairmanship of h.e. dr. mohammed rashid ahmed al hamli, and the membership of: • mubarak hamad m

In [60]:
RAG_QUESTION_GENERATOR_PROMPT = PromptTemplate(
    template= """
You are an AI assistant specialized in generating questions for Retrieval-Augmented Generation (RAG) systems. Your task is to analyze a given document and create 10 diverse questions that would effectively test a RAG system's ability to retrieve and synthesize information from this document.

Guidelines:
1. Thoroughly analyze the entire document.
2. Generate exactly 10 questions that cover various aspects and levels of complexity within the document's content.
3. Create questions that specifically target:
   a. Key facts and information
   b. Main concepts and ideas
   c. Relationships between different parts of the content
   d. Potential applications or implications of the information
   e. Comparisons or contrasts within the document
4. Ensure questions require answers of varying lengths and complexity, from simple retrieval to more complex synthesis.
5. Include questions that might require combining information from different parts of the document.
6. Frame questions to test both literal comprehension and inferential understanding.
7. Avoid yes/no questions; focus on open-ended questions that promote comprehensive answers.
8. Consider including questions that might require additional context or knowledge to fully answer, to test the RAG system's ability to combine retrieved information with broader knowledge.
9. Number the questions from 1 to 10.
10. Output only the ten questions, without any additional text, explanations, or answers.

Document:
{document_content}

Generate 10 questions optimized for testing a RAG system based on this document:
""",
    input_variables=["document_content"],
)

question_generation = RAG_QUESTION_GENERATOR_PROMPT | llm

In [61]:


#summary_text = final_output['executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities']

for index, content in final_output.items():
    response = question_generation.invoke({"document_content": content })
    print(index)
    print(content)
    print(response.content)

executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities
executive council resolution no. (11) of 2023 concerning the reformation of the board of trustees of mohamed bin zayed university for humanities having reviewed chairman of the executive council resolution no. (27) of 2020 concerning the formation of the board of trustees of mohamed bin zayed university for humanities. the executive council has decided the following: 1. the board of trustees of mohamed bin zayed university for humanities shall be reformed under the chairmanship of h.e. dr. mohammed rashid ahmed al hamli, and the membership of: • mubarak hamad mubarak al muhairi - vice chairman • mohammed hamza hassan al qassim. • maryam eid khamis al muhairi • abdullah aqueeda ali al muhairi • dr. omar habtoor theeb al-derei. • ghanem sultan ahmed al suwaidi. 2. the membership term of the board shall be three renewable years. 3. this resolutio